In [0]:
%run /Users/georgiavieiras@gmail.com/Ecommerce/Ambiente/Conexão_Ambiente

In [0]:
%sql
-- Criar catalogo e schema
CREATE CATALOG IF NOT EXISTS ecommerce;
CREATE SCHEMA IF NOT EXISTS bronze;

In [0]:
# Limpando widgets existentes (boa prática em notebooks reutilizáveis)
dbutils.widgets.removeAll()

# Widgets de fonte
dbutils.widgets.text("source_table", "public.produtos")

# Widgets de destino
dbutils.widgets.text("target_table", "produtos")
dbutils.widgets.text("catalog", "ecommerce")
dbutils.widgets.text("schema", "bronze")

# Lendo valores
source_table = dbutils.widgets.get("source_table")
target_table = dbutils.widgets.get("target_table")
catalog = dbutils.widgets.get("catalog")
schema = dbutils.widgets.get("schema")


In [0]:
df_source = spark.read.jdbc(
    url=jdbc_url,
    table=source_table,
    properties=connection_properties
)

In [0]:
from pyspark.sql.functions import lit, current_timestamp

# Leitura do Dataframe da camada bronze
df = spark.read.table(f"{catalog}.{schema}.{target_table}")

# Adicionando colunas de controle
df_bronze = (
    df_source
    .withColumn("data_source", lit(source_table))
    .withColumn("ingestion_ts", current_timestamp())
)

# Salvando como tabela Delta
df_bronze.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(f"{catalog}.{schema}.{target_table}")